In [ ]:
"""
"Zombie" members - people who sign up and pay for a membership online but
never come in to activate it

New accounts get a "User Warning" filled in with a message informing people to check
IDs on their first visit. This warning isn't ALWAYS cleared, but its presence is a
good indicator the user hasn't actually used the tool library yet.  We can make that
signal even more reliable by excluding users with that warning if they've checked
something out.

TODO: "Shop Use Only" members who have checked out tools?
"""
import pandas as pd

DATA_DIR = "~/chtl/data/processed/"
members = pd.read_pickle(f'{DATA_DIR}users.pkl')

members[:10]

In [ ]:
ACTIVE_MEMBERSHIP_TYPES = {
    'regular',
    'Flexible',
    'Standard (Annual)',
    'Standard (Monthly)',
    'Sustaining (Annual',
}
active_members_with_activation_warnings = members[
    members['Current Membership Type'].isin(ACTIVE_MEMBERSHIP_TYPES)][
    members['User Warning'].str.contains('This member is new', na=False)]

active_members_with_activation_warnings[:10]

In [ ]:
from datetime import datetime, timedelta

THIRTY_DAYS_AGO = datetime.now() - timedelta(days=20)

# 'First Membership Started' is the original membership if they switched types (e.g. the legacy 'regular' to
# one of the self-signup memberships)
thirty_day_zombies = active_members_with_activation_warnings[
    active_members_with_activation_warnings['First Membership Started'] < THIRTY_DAYS_AGO]

thirty_day_zombies[:10]

In [ ]:
loans = pd.read_pickle(f'{DATA_DIR}loans.pkl')

member_ids_with_loans = set(loans['Membership ID'])

zombie_members_including_flexible = thirty_day_zombies[~thirty_day_zombies['Membership ID'].isin(member_ids_with_loans)]

zombie_members_including_flexible[:10]

In [ ]:
transactions = pd.read_pickle(f'{DATA_DIR}transactions.pkl')

zombie_members = []
# Exclude Flexible members who paid $0. Not a huge deal since they didn't pay anything.
for ix, m in zombie_members_including_flexible.iterrows():
    member_txns = transactions[transactions['Member ID'] == m['Membership ID']]
    # The structure of txns is weird. A 'Membership Fee' will be present for non-$0 memberships.
    if len(member_txns[member_txns['Transaction Type'] == 'Membership Fee']) > 0:
        zombie_members.append(m)

pd.DataFrame(zombie_members)